In [219]:
import pandas as pd
from math import sqrt
import datetime
import numpy as np

In [2]:
usersDF = pd.read_csv('../../../Downloads/myanimelist/users_cleaned.csv')
animesDF = pd.read_csv('../../../Downloads/myanimelist/anime_cleaned.csv')
scoresDF = pd.read_csv('../../../Downloads/myanimelist/scores_sample.csv')

In [3]:
print(usersDF.shape, animesDF.shape, scoresDF.shape)

(108711, 17) (6668, 33) (5000000, 4)


In [4]:
animesDF.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0,2012.0


In [5]:
scoresDF.head()

,username,anime_id,my_score,my_status
0,goddrizzit,2251,9,2
1,missmmg,597,9,2
2,Shogen,33,7,4
3,FadeKyun,20,3,4
4,LuisGerardoMoret,430,9,2


In [6]:
'''
gets all user's scores
'''
def get_viewer(scoresDF, viewer):
    return scoresDF.loc[(scoresDF['username'] == viewer)]

get_viewer(scoresDF, 'Faultydoombot').head()

,username,anime_id,my_score,my_status
149298,Faultydoombot,28121,9,2
283073,Faultydoombot,17265,8,3
568416,Faultydoombot,2904,10,2
749173,Faultydoombot,9367,7,2
780235,Faultydoombot,3407,9,4


In [148]:
def sim_pearson(scores, anime1_id, anime2_id):
    '''Return the Pearson correlation coefficient for anime1 and anime2.'''
    scores_anime1 = scores[scores.anime_id == anime1_id]
    scores_anime2 = scores[scores.anime_id == anime2_id]
    # get the list of shared viewers
    shared_viewers = 0
    scores1 = np.array([])
    scores2 = np.array([])
    count = 0
    n = scores_anime1.shape[0]
    for index, row in scores_anime1.iterrows():
        viewer = get_viewer(scores_anime2, row['username'])
        if not viewer.empty:
            shared_viewers += 1
            scores1 = np.append(scores1, row['my_score'])
            scores2 = np.append(scores2, int(viewer['my_score']))
        count += 1
        if count == n:
            print('Viewers cheked: {} out of {} :: Shared: {} :: {}'.format(count, n, shared_viewers, datetime.datetime.now()))
            
    # if no ratings in common, return 0
    if shared_viewers == 0:
        return 0

    # add up all the preferences
    sum1 = np.sum(scores1)
    sum2 = np.sum(scores2)
    
    # sum up the squares
    sum1Sq = np.sum(scores1**2)
    sum2Sq = np.sum(scores2**2)
    
    # sum up the products
    #pSum = sum([score1*score2 for score1, score2 in zip(scores1, scores2)])
    pSum = np.dot(scores1, scores2)
    # calculate Pearson score
    num = pSum - (sum1 * sum2)/shared_viewers
    den = sqrt((sum1Sq - pow(sum1, 2)/shared_viewers) * (sum2Sq - pow(sum2, 2)/shared_viewers))
    if den == 0:
        return 0
    else:
        return num/den # between(0, 1)

In [149]:
sim_pearson(scoresDF, 1735, 20) #comparing Naruto and Naruto: Shippuuden

Viewers cheked: 10136 out of 10136 :: Shared: 2436 :: 2020-02-01 17:27:11.615891


0.6956989060013002

In [150]:
anime_ids = np.sort(scoresDF.anime_id.unique())
anime_ids

array([    1,     5,     6, ..., 37814, 37859, 37860])

In [208]:
'''
builds table of similarity of the shows
a[i][j] - similarity of the shows with ids: anime_ids[i-1] and anime_ids[j-1]
'''
def get_similarity_table(scores, anime_ids):
    similarity_table = np.zeros((len(anime_ids)+1, len(anime_ids)+1))
    similarity_table[1:, 0] = anime_ids
    similarity_table[0, 1:] = anime_ids
    for anime1_id in anime_ids:
        for anime2_id in anime_ids:
            if anime1_id != anime2_id:
                similarity = sim_pearson(scores, anime1_id, anime2_id)
                x = np.where(similarity_table == anime1_id)[0][1]
                y = np.where(similarity_table == anime2_id)[0][1]
                similarity_table[x][y] = similarity
                similarity_table[y][x] = similarity
        np.save('similarity_table', similarity_table)
    return similarity_table

In [209]:
get_similarity_table(scoresDF, anime_ids[:10])

Viewers cheked: 8360 out of 8360 :: Shared: 939 :: 2020-02-01 17:51:37.280641
Viewers cheked: 8360 out of 8360 :: Shared: 1083 :: 2020-02-01 17:51:53.898801
Viewers cheked: 8360 out of 8360 :: Shared: 290 :: 2020-02-01 17:52:06.285978
Viewers cheked: 8360 out of 8360 :: Shared: 28 :: 2020-02-01 17:52:17.025812
Viewers cheked: 8360 out of 8360 :: Shared: 229 :: 2020-02-01 17:52:30.319924
Viewers cheked: 8360 out of 8360 :: Shared: 387 :: 2020-02-01 17:52:43.231303
Viewers cheked: 8360 out of 8360 :: Shared: 70 :: 2020-02-01 17:52:53.208456
Viewers cheked: 8360 out of 8360 :: Shared: 197 :: 2020-02-01 17:53:03.496094
Viewers cheked: 8360 out of 8360 :: Shared: 588 :: 2020-02-01 17:53:17.762549
Viewers cheked: 3873 out of 3873 :: Shared: 939 :: 2020-02-01 17:53:29.825967
Viewers cheked: 3873 out of 3873 :: Shared: 581 :: 2020-02-01 17:53:39.579234
Viewers cheked: 3873 out of 3873 :: Shared: 200 :: 2020-02-01 17:53:45.002588
Viewers cheked: 3873 out of 3873 :: Shared: 20 :: 2020-02-01 17:5

array([[ 0.        ,  1.        ,  5.        ,  6.        ,  7.        ,
         8.        , 15.        , 16.        , 17.        , 18.        ,
        19.        ],
       [ 1.        ,  0.        ,  0.53953193,  0.24937932,  0.2737769 ,
         0.29842453,  0.02348753,  0.32983719,  0.09944638,  0.16381224,
         0.29334198],
       [ 5.        ,  0.53953193,  0.        ,  0.2346193 ,  0.43994489,
         0.09298749,  0.19581978,  0.35608832,  0.15863545,  0.37783181,
         0.11249033],
       [ 6.        ,  0.24937932,  0.2346193 ,  0.        ,  0.289953  ,
         0.32640795,  0.24757952,  0.21272643,  0.28037038,  0.30038744,
         0.1660099 ],
       [ 7.        ,  0.2737769 ,  0.43994489,  0.289953  ,  0.        ,
         0.59605165,  0.09895773,  0.41328327,  0.29413492,  0.25360885,
         0.14709618],
       [ 8.        ,  0.29842453,  0.09298749,  0.32640795,  0.59605165,
         0.        ,  0.68543457, -0.38425759,  0.09380832,  0.60442332,
         0.420

In [210]:
'''
loads user's scores from MAL xml file converted to csv
'''
def load_user_list(path):
    userDF = pd.read_csv(path)
    return userDF[['series_animedb_id', 'my_score']][userDF['my_score']!=0]

In [211]:
cmlimmDF = load_user_list('../../../Downloads/animelist_cmlimm.csv')

In [212]:
'''
find user's shows that exist in MAL dataset, as some shows aired after dataset was created
'''
def user_shows_in_db(userDF, animesDF):
    return userDF[userDF[['series_animedb_id']].isin(animesDF.anime_id.unique()).any(1)]

In [213]:
np.array(user_shows_in_db(cmlimmDF, animesDF))

array([[32977,     7],
       [22147,     9],
       [35015,     7],
       [ 2251,    10],
       [ 3901,     8],
       [35965,     7],
       [28805,     7],
       [32664,     6],
       [31964,     8],
       [33486,    10],
       [35262,     5],
       [36456,     9],
       [33929,     8],
       [35459,     8],
       [16592,     6],
       [35849,     8],
       [30346,     7],
       [ 6746,     8],
       [ 9314,     7],
       [18507,     7],
       [22265,     8],
       [26213,     8],
       [19671,     7],
       [35198,     8],
       [33731,     5],
       [ 2890,     7],
       [18229,     7],
       [20583,     9],
       [28891,     9],
       [32935,     9],
       [25303,     7],
       [35806,     7],
       [27989,    10],
       [31988,    10],
       [34204,     7],
       [31665,     7],
       [31096,     7],
       [30415,     8],
       [33241,     6],
       [10357,     9],
       [ 5680,     6],
       [ 9617,     6],
       [ 7791,     6],
       [ 97

In [214]:
test_sim_table = Out[152]
test_user_scores = np.array([[1, 9],[6, 10], [15, 1]])

In [215]:
'''
finds top matches based on shows similarities
'''
def top_matches(similarity_table, user_show_scores):
    result = np.empty((0,2), int)
    user_shows = user_show_scores[:, 0]
    user_scores = user_show_scores[:, 1]
    to_delete = []
    for show in user_shows:
        y = np.where(similarity_table == show)[0][1]
        to_delete.append(y)
    similarity_table = np.delete(similarity_table, to_delete, 1)
    to_delete = []
    for show in similarity_table[1:, 0]:
        if show not in user_shows:
            x = np.where(similarity_table == show)[0][1]
            to_delete.append(x)
    similarity_table = np.delete(similarity_table, to_delete, 0)
    for column in similarity_table.T[1:]:
        result = np.append(result, [[column[0], np.dot(column[1:], user_scores)/np.sum(column[1:])]], axis=0)
    return result[result[:,1].argsort()][::-1]

In [216]:
top_matches(test_sim_table, test_user_scores)

array([[ 7.        ,  8.242918  ],
       [19.        ,  7.84421594],
       [ 5.        ,  7.62682604],
       [16.        ,  6.1643165 ],
       [18.        ,  5.25746322],
       [ 8.        ,  5.06410876],
       [17.        ,  4.77553594]])

In [217]:
def show_names(matches, animesDF):
    for show in matches:
        print(animesDF[animesDF['anime_id']==show[0]]['title'].values[0])

In [218]:
show_names(top_matches(test_sim_table, test_user_scores), animesDF)

Witch Hunter Robin
Monster
Cowboy Bebop: Tengoku no Tobira
Hachimitsu to Clover
Initial D Fourth Stage
Beet the Vandel Buster
Hungry Heart: Wild Striker
